# Derivative of traits_i with respect to traits_k

Here I will be testing my solution for
$\frac{ \partial \mathbf{V_{i,t+1}} }{ \partial \mathbf{V_{k,t}} }$
(see below)
by calculating the Jacobian using the `theano` package
and comparing those results to my solution.




## Importing packages and setting options

In [1]:
%env OMP_NUM_THREADS=4
%env THEANO_FLAGS='openmp=True'
import sympy
import theano
theano.config.cxx = ""
import theano.tensor as T
import numpy as np
import pandas as pd
from tqdm import tqdm
import math
pd.options.display.max_columns = 10

env: OMP_NUM_THREADS=4
env: THEANO_FLAGS='openmp=True'


## Equations

__Notes:__

- ${}^\text{T}$ represents transpose.
- Elements in __bold__ are matrices
- Multiplication between matrices is always matrix multiplication, not
  element-wise
  

The equations for (1) traits for species $i$ at time $t+1$ ($\mathbf{V}_{i,t+1}$)
and (2) the partial derivative of species $i$ traits with respect
to species $k$ traits (where $k \ne i$)
are as follows:

\begin{align}
\mathbf{V}_{i,t+1} &= \mathbf{V}_{i,t} + 2 ~ \sigma_i^2
    \left[
        \left(
            N_{k,t} ~ \textrm{e}^{-\mathbf{V}_{k,t} \mathbf{D} \mathbf{V}_{k,t}^\textrm{T}} +
            \mathbf{\Phi}_{i,t}
        \right)
        \left(
            \alpha_0 ~ \textrm{e}^{-\mathbf{V}_{i,t}
            \mathbf{V}_{i,t}^\textrm{T}} ~ \mathbf{V}_{i,t}
        \right)
        - f \mathbf{V}_{i,t} \mathbf{C}
    \right] \\
    \mathbf{\Phi}_{i,t} &= N_{i,t} + \sum_{j \ne i, j \ne k}^{n}{
        N_{j,t} ~ \textrm{e}^{- \mathbf{V}_{j,t} \mathbf{D}
        \mathbf{V}_{j,t}^{\textrm{T}}} } \\
    \frac{ \partial\mathbf{V}_{i,t+1} }{ \partial\mathbf{V}_{k,t}} &=
        -4 ~ \alpha_0 ~ \sigma^2 ~ N_{k,t} ~ 
        \mathbf{D} ~ \mathbf{V}_{k,t}^{\textrm{T}} ~
        \textrm{e}^{
                - \mathbf{V}_{k,t} \mathbf{D} \mathbf{V}_{k,t}^{\textrm{T}}
                - \mathbf{V}_{i,t} \mathbf{V}_{i,t}^{\textrm{T}}
            } ~
            \mathbf{V}_{i,t}
\end{align}


## Read CSV of simulated datasets

In [2]:
sims = pd.read_csv("simulated_data.csv")
sims.head()

,V1,V2,V3,V4,V5,...,f,a0,eta,r0,d
0,4.945110,2.869199,6.747126,6.142522,5.629532,...,0.068890,0.112113,-0.331150,1.422746,-0.091228
1,0.718846,1.220364,0.815571,0.868633,0.838021,...,0.309021,0.057579,0.094811,1.237047,0.003429
2,3.369285,1.912974,3.131174,0.046303,1.416252,...,0.118318,0.401410,-0.036977,1.746024,0.012160
3,0.373669,0.283873,0.237735,0.053632,0.062281,...,0.497286,0.499730,0.117188,0.669199,0.081612
4,3.562637,1.635016,5.724176,4.953962,1.060083,...,0.042638,0.307171,-0.467453,0.952351,0.051834


## Functions to compare methods

In [18]:
def automatic(i, k, N, V, D, f, a0, eta, s2):
    """Automatic differentiation using theano pkg"""
    Vi = V[i,:]
    Ni = N[i]
    Nk = N[k]
    C = np.zeros((3, 3)) + eta
    np.fill_diagonal(C,1.0)
    P = [np.exp(-1 * np.dot(np.dot(V[j,:], D), V[j,:].T)) * N[j] 
         for j in range(0, N.size) if j != i and j != k]
    P = np.sum(P) + Ni
    Vk_ = T.dvector('Vk_')
    Vhat = Vi + 2 * s2 * ( T.dot(Nk * T.exp(-1 * T.dot(T.dot(Vk_, D), Vk_.T)) + P, 
                              a0 * T.dot(T.exp(-1 * T.dot(Vi, Vi.T)), Vi)) -
                       f * T.dot(Vi, C) )
    J, updates = theano.scan(lambda i, Vhat, Vk_ : T.grad(Vhat[i], Vk_), 
                         sequences=T.arange(Vhat.shape[0]), non_sequences=[Vhat, Vk_])
    num_fun = theano.function([Vk_], J, updates=updates)
    out_array = num_fun(V[k,:]).T
    return out_array

In [19]:
def symbolic(i, k, N, V, D, a0, s2):
    """Symbolic differentiation using math"""
    Vi = V[i,:]
    Vi = Vi.reshape((1, 3))
    Vk = V[k,:]
    Vk = Vk.reshape((1, 3))
    Ni = N[i]
    Nk = N[k]
    ZZZ = np.exp(-1 * np.dot(np.dot(Vk, D), Vk.T) - np.dot(Vi, Vi.T))
    ZZZ = np.dot(np.dot(np.dot(D, Vk.T), ZZZ), Vi)
    dVhat = -4 * a0 * s2 * Nk * ZZZ
    return dVhat

In [20]:
def compare_methods(sim_i, s2 = 0.01, abs = False):
    """Compare answers from symbolic and automatic methods"""
    N = sims.loc[sim_i, [x.startswith("N") for x in sims.columns]].values
    V = sims.loc[sim_i, [x.startswith("V") for x in sims.columns]].values
    n, q = (N.size, int(V.size / N.size))
    V = V.reshape((n, q), order = 'F')
    f = sims.loc[sim_i,"f"]
    a0 = sims.loc[sim_i,"a0"]
    eta = sims.loc[sim_i,"eta"]
    D = np.zeros((q, q))
    np.fill_diagonal(D, sims.loc[sim_i,"d"])
    # r0 = sims.loc[sim_i,"r0"]  # don't need this one now
    diffs = np.empty((math.factorial(n) // math.factorial(n-2), 4))
    j = 0
    for i in range(0, n):
        for k in [x for x in range(0, n) if x != i]:
            num = automatic(i, k, N, V, D, f, a0, eta, s2)
            sym = symbolic(i, k, N, V, D, a0, s2)
            num = num.flatten()
            sym = sym.flatten()
            if abs:
                diff = num - sym
            else:
                diff = (num - sym) / sym
                if np.any(sym == 0):
                    for l in [x for x in range(0, diff.size) if sym[x] == 0]:
                        diff[l] = num[l];
            diffs[j, 0] = i
            diffs[j, 1] = k
            diffs[j, 2] = diff.min()
            diffs[j, 3] = diff.max()
            j += 1
    return diffs

### Example of using `compare_methods`:

In [21]:
diffs = compare_methods(0)
print(diffs[:,2].min())
print(diffs[:,3].max())

-3.3988825280693052e-15
1.3113329849740812e-15


## Comparing methods

This takes ~5-6 minutes.

In [22]:
n_per_rep = math.factorial(4) // math.factorial(4-2)
diffs = np.empty((int(n_per_rep * 100), 4))

In [23]:
for rep in tqdm(range(100)):
    diffs_r = compare_methods(rep)
    diffs[(rep * n_per_rep):((rep+1) * n_per_rep),:] = diffs_r

100%|██████████| 100/100 [10:34<00:00,  6.34s/it]


## The results
They appear to have extremely similar values, similar enough to make me quite comfortable saying that my symbolic solution works.

In [24]:
print(diffs[:,2].min())
print(diffs[:,3].max())

-0.00039416633819471815
8.678295582747549e-05


## Write output to file

To make sure the R version works, too, I'm writing to a CSV file the output from the symbolic version on the 100 datasets.

In [39]:
n = np.sum([x.startswith("N") for x in sims.columns])
q = int(np.sum([x.startswith("V") for x in sims.columns]) / n)
s2 = 0.01
n_perms = math.factorial(n) // math.factorial(n-2)
# Output array
results = np.zeros((100, n_perms * q * q))

for sim_i in range(100):
    
    # Fill info from data frame:
    N = sims.loc[sim_i, [x.startswith("N") for x in sims.columns]].values
    V = sims.loc[sim_i, [x.startswith("V") for x in sims.columns]].values
    V = V.reshape((n, q), order = 'F')
    a0 = sims.loc[sim_i,"a0"]
    D = np.zeros((q, q))
    np.fill_diagonal(D, sims.loc[sim_i,"d"])

    # Fill output array:
    j = 0
    for i in range(0, n):
        for k in [x for x in range(0, n) if x != i]:
            sym = symbolic(i, k, N, V, D, a0, s2)
            results[sim_i, (j*q*q):((j+1)*q*q)] = sym.flatten()
            j += 1

# Make sure the last row isn't zeros:
results[99, :]

array([-2.64267017e-07, -4.46146582e-07, -6.94742448e-08, -3.63466399e-07,
       -6.13619109e-07, -9.55531790e-08, -7.35092604e-08, -1.24101394e-07,
       -1.93251523e-08, -1.75653653e-07, -2.96545811e-07, -4.61783124e-08,
       -6.10548346e-07, -1.03075314e-06, -1.60509570e-07, -1.31053657e-06,
       -2.21250240e-06, -3.44532357e-07, -1.52004866e-07, -2.56620944e-07,
       -3.99611855e-08, -2.93869231e-08, -4.96122273e-08, -7.72564929e-09,
       -9.71448078e-08, -1.64003910e-07, -2.55387987e-08, -4.38690694e-03,
       -6.03364462e-03, -1.22027443e-03, -7.40615896e-03, -1.01862501e-02,
       -2.06011811e-03, -1.15329204e-03, -1.58620969e-03, -3.20802973e-04,
       -1.26036310e-03, -1.73347261e-03, -3.50586162e-04, -4.38085170e-03,
       -6.02531640e-03, -1.21859009e-03, -9.40345902e-03, -1.29332878e-02,
       -2.61569274e-03, -1.09067657e-03, -1.50008991e-03, -3.03385678e-04,
       -2.10859227e-04, -2.90010629e-04, -5.86531986e-05, -6.97040619e-04,
       -9.58692637e-04, -

In [43]:
np.savetxt('results/dVi_dVk.csv', results, delimiter=',')